# LRS Slitless trace plotting notebook

Author: Jonathan Aguilar (jaguilar@stsci.edu)

This notebook visualizes the spectral traces for LRS Slitless observations, for a given orientation of the telescope (this is an early version, so the tools for trace visualization may evolve over time). For a more general introduction to WHIPPOT, see [WHIPPOT_tutorial](WHIPPOT_tutorial).

This notebook was originally designed for observations that use target acquisition on one star and then slew to another. LRS Slitless does not need this, so users should use the same coordinates for their ACQ target as for their SCI target. 

The user provides the RA and Dec coordinates for the target acquisition (ACQ) star and science (SCI) star, and a telescope roll angle. These tools will then compute a) the "Special Requirement Offset" X and Y values needed to place the SCI star at the reference position, and b) the final coordinates of the SCI and ACQ stars. Coordinates are given in IDL (https://jwst-docs.stsci.edu/jwst-observatory-hardware/jwst-target-observability-and-observatory-coordinate-system/jwst-instrument-ideal-coordinate-systems#gsc.tab=0).

The user can also provide a list of extra targets in the field in the format specified below, and their IDL coordinates will also be computed. As such, this notebook is useful not just for planning target acquisition, but also for predicting to high precision the final positions of astronomical bodies in the field of view.

## Setup ##

  1. Navigate in a terminal to the directory where you have downloaded the module. This should contain the `setup.py` file.
  2. Activate your desired python environment.
  3. Run `pip install .` (or `pip install -e.` for developing)
  4. Copy this notebook to your working directory, and run your jupyter kernel in the appropriate environment.

In [1]:
# for development purposes
%load_ext autoreload
%autoreload 2

In [2]:
# for your system, choose an appropriate plotting backend to make pop-up window plots
# (currently, inline plots show up in the log but not directly in the notebook).
# %matplotlib throws an error if you run it first, so do inline first.
%matplotlib inline
%matplotlib osx

In [4]:
from whippot import whippot_tools
from whippot.modes import miri_mrs_tools
# if the library is not installed replace the line above with the line below 
# when `whippot_tools.py` is in the same folder as the notebook
# import whippot_tools

## An example of a real field

The source coordinates below are taken from a real commissioning activity, where we needed to check that the traces wouldn't overlap.

In [5]:
# some 
sources = {
    'SCI': whippot_tools.SkyCoord("05 24 20.7552 -70 05 1.60", frame='icrs', unit=("hourangle","degree")),
    'a': whippot_tools.SkyCoord("05 24 26.33969 -70 05 22.3545", frame='icrs', unit=("hourangle","degree")),
    'b': whippot_tools.SkyCoord("05 24 28.67861 -70 05 24.4484", frame='icrs', unit=("hourangle","degree")),
    'c': whippot_tools.SkyCoord("05 24 36.22460 -70 05 28.1876", frame='icrs', unit=("hourangle","degree")),
    'd': whippot_tools.SkyCoord("05 24 25.608 -70 05 01.66", frame='icrs', unit=("hourangle","degree")),
}

In [6]:
# format the sources for easy copy-pasting
for s, c in sources.items():
    print(f"{s}: ({c.ra.deg}, {c.dec.deg})")

SCI: (81.08648, -70.08377777777777)
a: (81.10974870833334, -70.08954291666666)
b: (81.11949420833332, -70.09012455555555)
c: (81.15093583333332, -70.09116322222222)
d: (81.1067, -70.08379444444444)


In [38]:
from astropy import units
initial_values={
    'instr': 'miri',
    'sci_aper': 'mirifu_channel1a', 
    'pa': 290.,
    'sci_ra': sources['SCI'].ra.deg, 'sci_dec': sources['SCI'].dec.deg,
}
# add a multi-line string of the other stars, copied from the cell above
initial_values['other_stars'] = """
a: (81.10974870833334, -70.08954291666666)
d: (81.1067, -70.08379444444444)
"""

## Use the LRS Slitless-specific ComputePositions class

The ComputePositions class in `miri_lrs_slitless` is a subclass of the general `ComputePositions` class that adds LRS-specific overlays to the plots.

In [72]:
a = []
a.append(1)
a
a.append([2, 3])
a

[1, [2, 3]]

In [70]:
set([i.AperType for j, i in cp1.instr.apertures.items()])

{'COMPOUND', 'FULLSCA', 'OSS', 'ROI', 'SLIT', 'SUBARRAY'}

In [99]:
# cp1 = miri_mrs_tools.ComputePositions(initial_values=initial_values)
cp1 = whippot_tools.ComputePositions(initial_values=initial_values)
# fig = cp1.plot_scene()
cp1.ui


In [89]:
cp1.aperture.AperType

'ROI'

In [55]:
import matplotlib as mpl
from matplotlib import pyplot as plt

fig, ax = plt.subplots()
aper = cp1.instr['MIRIFU_CHANNEL1A']

aper.plot(frame='idl', ax=ax)

cp1.instr['MIRIFU_1ASLICE01'].plot(frame='idl', ax=ax)

In [95]:
c = whippot_tools.widgets.Checkbox()
c.observe?

Signature:
c.observe(
    handler: 't.Callable[..., t.Any]',
    names: 'Sentinel | str | t.Iterable[Sentinel | str]' = traitlets.All,
    type: 'Sentinel | str' = 'change',
) -> 'None'
Docstring:
Setup a handler to be called when a trait changes.

This is used to setup dynamic notifications of trait changes.

Parameters
----------
handler : callable
    A callable that is called when a trait changes. Its
    signature should be ``handler(change)``, where ``change`` is a
    dictionary. The change dictionary at least holds a 'type' key.
    * ``type``: the type of notification.
    Other keys may be passed depending on the value of 'type'. In the
    case where type is 'change', we also have the following keys:
    * ``owner`` : the HasTraits instance
    * ``old`` : the old value of the modified trait attribute
    * ``new`` : the new value of the modified trait attribute
    * ``name`` : the name of the modified trait attribute.
names : list, str, All
    If names is All, the handler